Based on [official docs](https://www.tensorflow.org/guide/eager)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib import eager as tfe 

tf.enable_eager_execution()

In [2]:
tf.executing_eagerly()

True

In [3]:
x = [[2.]]
m = tf.matmul(x, x)
m.numpy()

array([[4.]], dtype=float32)

In [4]:
x = tfe.Variable([[3.0]])
with tf.GradientTape(persistent=True) as g:
    g.watch(x)
    y = x * x
    z = y * y
print(g.gradient(z, x))
print(g.gradient(y, x))

tf.Tensor([[108.]], shape=(1, 1), dtype=float32)
tf.Tensor([[6.]], shape=(1, 1), dtype=float32)


In [5]:
# A toy dataset of points around 3 * x + 2
NUM_EXAMPLES = 1000
training_inputs = tf.random_normal([NUM_EXAMPLES])
noise = tf.random_normal([NUM_EXAMPLES])
training_outputs = training_inputs * 3 + 2 + noise

def prediction(input, weight, bias):
    return input * weight + bias

# A loss function using mean-squared error
def loss(weights, biases):
    error = prediction(training_inputs, weights, biases) - training_outputs
    return tf.reduce_mean(tf.square(error))

# Return the derivative of loss with respect to weight and bias
def grad(weights, biases):
    with tf.GradientTape() as tape:
        loss_value = loss(weights, biases)
    return tape.gradient(loss_value, [weights, biases])

train_steps = 200
learning_rate = 0.01
# Start with arbitrary values for W and B on the same batch of data
W = tfe.Variable(5.)
B = tfe.Variable(10.)

print("Initial loss: {:.3f}".format(loss(W, B)))

for i in range(train_steps):
    dW, dB = grad(W, B)
    W.assign_sub(dW * learning_rate)
    B.assign_sub(dB * learning_rate)
    if i % 20 == 0:
        print("Loss at step {:03d}: {:.3f}".format(i, loss(W, B)))

print("Final loss: {:.3f}".format(loss(W, B)))
print("W = {}, B = {}".format(W.numpy(), B.numpy()))

Initial loss: 67.574
Loss at step 000: 64.981
Loss at step 020: 29.918
Loss at step 040: 14.082
Loss at step 060: 6.921
Loss at step 080: 3.678
Loss at step 100: 2.208
Loss at step 120: 1.540
Loss at step 140: 1.237
Loss at step 160: 1.099
Loss at step 180: 1.036
Final loss: 1.008
W = 3.1109156608581543, B = 2.1451125144958496


In [6]:
def square(x, y):
    return tf.multiply(x, y)

grad = tfe.gradients_function(square)
print(square(3., 3.).numpy())  # => 9.0
print(grad(3., 3.))    # => [6.0]

9.0
[<tf.Tensor: id=7388, shape=(), dtype=float32, numpy=3.0>, <tf.Tensor: id=7389, shape=(), dtype=float32, numpy=3.0>]


In [7]:
@tf.custom_gradient
def log1pexp(x):
    e = tf.exp(x)
    def grad(dy):
        return dy * (1 - 1 / (1 + e))
    return tf.log(1 + e), grad

grad_log1pexp = tfe.gradients_function(log1pexp)
print(grad_log1pexp(0.0))
print(grad_log1pexp(1.0))
print(grad_log1pexp(10.0))
print(grad_log1pexp(100.0))

[<tf.Tensor: id=7401, shape=(), dtype=float32, numpy=0.5>]
[<tf.Tensor: id=7412, shape=(), dtype=float32, numpy=0.7310586>]
[<tf.Tensor: id=7422, shape=(), dtype=float32, numpy=0.9999546>]
[<tf.Tensor: id=7433, shape=(), dtype=float32, numpy=1.0>]


In [8]:
tfe.num_gpus()

0